In [56]:
import psycopg2
from config import config

In [57]:
def connect():
    '''Connect to the PostGresQL Database Server'''
    conn = None
    try:
        params = config()
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
        db_version = cur.fetchone()
        print(db_version)
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

if __name__=='__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.5.25 on x86_64-apple-darwin21.1.0, compiled by Apple clang version 13.0.0 (clang-1300.0.29.3), 64-bit',)
Database connection closed.


In [58]:
def get_info(query):
    '''this function queries the database and find the information specified in the parameter'''
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute(query)
        results = cur.fetchall()
        return results
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [59]:
# what is the email address for the customer with the name Nancy Thomas?
get_info("SELECT email FROM customer WHERE first_name = 'Nancy' AND last_name = 'Thomas'")

[('nancy.thomas@sakilacustomer.org',)]

In [60]:
# give a customer the description of the movie "Outlaw Hanky"
get_info("SELECT description FROM film WHERE title = 'Outlaw Hanky';")

[('A Thoughtful Story of a Astronaut And a Composer who must Conquer a Dog in The Sahara Desert',)]

In [61]:
# get the customer number for the customer who lives at 259 Ipoh Drive
get_info("SELECT phone FROM address WHERE address = '259 Ipoh Drive';")

[('419009857119',)]

In [62]:
# get more information about the customer who lives at 259 Ipoh Drive
get_info("""SELECT 
            c.first_name, c.last_name, c.email, a.address, a.phone 
        FROM customer c 
            LEFT JOIN 
                    address a 
                ON c.address_id = a.address_id 
            WHERE a.address = '259 Ipoh Drive';""")

[('Jesus',
  'Mccartney',
  'jesus.mccartney@sakilacustomer.org',
  '259 Ipoh Drive',
  '419009857119')]

In [63]:
# what are the customer ids of the 10 first customers who created a payment?
get_info("SELECT customer_id from payment ORDER BY payment_date ASC LIMIT 10;")

[(416,), (516,), (239,), (592,), (49,), (264,), (46,), (481,), (139,), (595,)]

In [64]:
# what are the titles of the 5 shortest movies?
get_info("SELECT title, length FROM film ORDER BY length ASC LIMIT 5;")

[('Iron Moon', 46),
 ('Kwai Homeward', 46),
 ('Alien Center', 46),
 ('Ridgemont Submarine', 46),
 ('Labyrinth League', 46)]

In [65]:
# how many movies can a customer watch that are less than or equal to 50 minutes long?
get_info("SELECT COUNT(*) FROM film WHERE length <= 50;")

[(37,)]

In [66]:
# how many payment transactions were greater thn $5.00?
get_info("SELECT COUNT(amount) FROM payment WHERE amount > 5;")

[(3618,)]

In [67]:
# how many actors have a first name that starts with the letter 'P'?
get_info("SELECT COUNT(*) FROM actor WHERE first_name ILIKE 'p%';")

[(5,)]

In [68]:
# how many unique districts are our customers from?
get_info("SELECT COUNT(DISTINCT(district)) FROM address;")

[(378,)]

In [69]:
# retrieve the list of names for those distinct districts from the previous query
get_info("""SELECT DISTINCT(district) FROM address;""")

[('Asuncin',),
 ('Yerevan',),
 ('Nagasaki',),
 ('Nonthaburi',),
 ('Oriental',),
 ('Carabobo',),
 ('Bursa',),
 ('Cagayan Valley',),
 ('Mwanza',),
 ('West Bengali',),
 ('Jiangxi',),
 ('Daugavpils',),
 ('Anhalt Sachsen',),
 ('Shandong',),
 ('Anzotegui',),
 ('Lombardia',),
 ('Sinaloa',),
 ('Asir',),
 ('Andhra Pradesh',),
 ('Nampula',),
 ('Khartum',),
 ('Nord-Ouest',),
 ('Scotland',),
 ('Batman',),
 ('Baijeri',),
 ('Basel-Stadt',),
 ('Krasnojarsk',),
 ('Ahal',),
 ('Zanzibar West',),
 ('Veracruz',),
 ('Bretagne',),
 ('Cheju',),
 ('Osaka',),
 ('Botosani',),
 ('Missouri',),
 ('Aden',),
 ('North West',),
 ('St Thomas',),
 ('Adygea',),
 ('Zufar',),
 ('Pernambuco',),
 ('Henan',),
 ('Esfahan',),
 ('Songkhla',),
 ('Sousse',),
 ('Jalisco',),
 ('Baja California',),
 ('Southern Mindanao',),
 ('al-Qadarif',),
 ('Sucre',),
 ('Cear',),
 ('Central Java',),
 ('Pays de la Loire',),
 ('Sumqayit',),
 ('al-Sharqiya',),
 ('Mie',),
 ('Central Serbia',),
 ('Rajasthan',),
 ('Illinois',),
 ('Paran',),
 ('al-Manama'

In [70]:
# how many films are R-rated and have a replacement cost between $5 and $15?
get_info("""SELECT COUNT(*) FROM film WHERE rating = 'R' AND replacement_cost BETWEEN 5 AND 15;""")

[(52,)]

In [71]:
# how many films have the word Truman somewhere in the title?
get_info("""SELECT COUNT(title) FROM film WHERE title ILIKE '%truman%';""")

[(5,)]

In [72]:
# we have two staff members with staff ids 1 and 2. A bonus will be given to the staff member
#  who handled the most transactions. how many payments did each staff member handle and who gets the bonus?
get_info("""
        SELECT staff_id, COUNT(amount) FROM payment GROUP BY staff_id;
""")

[(2, 7304), (1, 7292)]

In [73]:
# corporate hq is conducting a study on the relationship  between replacement cost and a movie rating.
# what is the average replacement cost per rating?
get_info("""
        SELECT rating, ROUND(AVG(replacement_cost), 2) FROM film GROUP BY rating;
""")

[('R', Decimal('20.23')),
 ('PG', Decimal('18.96')),
 ('PG-13', Decimal('20.40')),
 ('NC-17', Decimal('20.14')),
 ('G', Decimal('20.12'))]

In [74]:
# we are running a promotiom to reward 5 customers with coupons.
# what are the customer ids of the 5 customers who have the most payments?
get_info(
    """
        select customer_id, sum(amount) from payment group by customer_id order by sum(amount) desc limit 5;
    """
)

[(148, Decimal('211.55')),
 (526, Decimal('208.58')),
 (178, Decimal('194.61')),
 (137, Decimal('191.62')),
 (144, Decimal('189.60'))]

In [75]:
# we are launching a platinum service for our most loyal customers. we will assidn a platinum membership to
# the customers who have made 40 or more payments. what customer_ids are eligible for platinum status?
get_info(
    """
    SELECT customer_id FROM payment GROUP BY customer_id HAVING COUNT(*) >= 40;
    """)

[(526,), (148,), (144,)]

In [76]:
# what are the customer ids of customers who have spent more than 100 in payment transactions with staff_id 2?
get_info(
    """
    SELECT customer_id, SUM(amount) 
    FROM payment 
    WHERE staff_id = 2 
    GROUP BY customer_id, staff_id 
    HAVING SUM(amount) > 100;
    """
    )

[(187, Decimal('110.81')),
 (526, Decimal('101.78')),
 (211, Decimal('108.77')),
 (148, Decimal('110.78')),
 (522, Decimal('102.80'))]

In [77]:
# return the customer id of customers who have spent at least 110 with the staff member with staff_id 2
get_info("SELECT customer_id from payment WHERE staff_id = 2 GROUP BY customer_id HAVING SUM(amount) > 110;")

[(187,), (148,)]

In [78]:
# how many films begin with the letter 'J'?
get_info("SELECT COUNT(*) from film WHERE title ILIKE 'j%';")

[(20,)]

In [79]:
# what customer has the highest customer id whose name starts with an 'E' and has an address ID lower than 500?
get_info("""
        SELECT MAX(customer_id), first_name, last_name
        FROM customer
        WHERE first_name ILIKE 'e%' 
        GROUP BY customer_id, first_name, last_name
        HAVING address_id < 500
        ORDER BY MAX(customer_id) DESC
        LIMIT 1;
""")

[(434, 'Eddie', 'Tomlin')]

_____________________________________________________________________________________________________________________

TIMESTAMPS

In [89]:
# during which months did payment occur?
get_info("""
        SELECT EXTRACT(MONTH FROM payment_date)
        FROM payment
        GROUP BY EXTRACT(MONTH FROM payment_date)
        ORDER BY EXTRACT(MONTH FROM payment_date) ASC;
""")

[(2.0,), (3.0,), (4.0,), (5.0,)]

In [92]:
# return back the full month name
get_info("""
    select distinct(to_char(payment_date, 'MONTH')) from payment;
""")

[('MAY      ',), ('APRIL    ',), ('MARCH    ',), ('FEBRUARY ',)]

In [99]:
# how many payments occurred on a monday?
# 0 = Sunday, 1 = Monday, 2 = Tuesday, 3 = Wednesday, 4 = Thursday, 5 = Friday, 6 = Saturday
# dow = day of week
get_info("""
    select count(extract(dow from payment_date)) from payment where extract(dow from payment_date) = 1;
""")

[(2948,)]

______________________________________________________________________________________________________________________

In [100]:
# mathematical operations
get_info("""
    select count(*) from payment where amount > (select avg(amount) from payment);
""")

[(7042,)]

In [102]:
# put 10% down on a replacement cost
# 10% = 0.1
get_info("""
    select 0.1 * replacement_cost as deposit from film;
""")

[(Decimal('1.499'),),
 (Decimal('1.999'),),
 (Decimal('1.599'),),
 (Decimal('1.299'),),
 (Decimal('2.099'),),
 (Decimal('1.299'),),
 (Decimal('1.899'),),
 (Decimal('2.699'),),
 (Decimal('2.299'),),
 (Decimal('1.799'),),
 (Decimal('2.899'),),
 (Decimal('2.199'),),
 (Decimal('2.499'),),
 (Decimal('1.699'),),
 (Decimal('2.299'),),
 (Decimal('1.999'),),
 (Decimal('2.199'),),
 (Decimal('2.399'),),
 (Decimal('1.099'),),
 (Decimal('2.399'),),
 (Decimal('1.499'),),
 (Decimal('2.799'),),
 (Decimal('2.099'),),
 (Decimal('2.399'),),
 (Decimal('1.799'),),
 (Decimal('1.099'),),
 (Decimal('0.999'),),
 (Decimal('1.999'),),
 (Decimal('1.599'),),
 (Decimal('1.599'),),
 (Decimal('1.299'),),
 (Decimal('1.699'),),
 (Decimal('1.199'),),
 (Decimal('2.799'),),
 (Decimal('1.699'),),
 (Decimal('1.199'),),
 (Decimal('1.599'),),
 (Decimal('2.999'),),
 (Decimal('2.499'),),
 (Decimal('1.299'),),
 (Decimal('2.899'),),
 (Decimal('2.599'),),
 (Decimal('1.099'),),
 (Decimal('2.299'),),
 (Decimal('1.799'),),
 (Decimal(

In [103]:
# addition
get_info("""
    select count(*) from payment 
        where amount + (
                        select avg(amount) from payment) > (select avg(amount) from payment);
""")

[(14572,)]

In [104]:
# division
get_info("""
    select count(*) from payment
        where amount / (select avg(amount) from payment) > (select avg(amount) from payment);
""")


[(0,)]

In [105]:
# power
get_info("""
    select count(*) from payment
        where amount ^ (select avg(amount) from payment) > (select avg(amount) from payment);
""")


[(11852,)]

In [106]:
get_info("SELECT 2 * 3;")

[(6,)]

In [107]:
get_info("SELECT 2 ^ 3;")

[(8.0,)]

______________________________________________________________________________________________________________________

In [108]:
# string functions and operators

# string concatenation
get_info("""
    select first_name || ' ' || last_name as full_name from customer;
""")

[('Jared Ely',),
 ('Mary Smith',),
 ('Patricia Johnson',),
 ('Linda Williams',),
 ('Barbara Jones',),
 ('Elizabeth Brown',),
 ('Jennifer Davis',),
 ('Maria Miller',),
 ('Susan Wilson',),
 ('Margaret Moore',),
 ('Dorothy Taylor',),
 ('Lisa Anderson',),
 ('Nancy Thomas',),
 ('Karen Jackson',),
 ('Betty White',),
 ('Helen Harris',),
 ('Sandra Martin',),
 ('Donna Thompson',),
 ('Carol Garcia',),
 ('Ruth Martinez',),
 ('Sharon Robinson',),
 ('Michelle Clark',),
 ('Laura Rodriguez',),
 ('Sarah Lewis',),
 ('Kimberly Lee',),
 ('Deborah Walker',),
 ('Jessica Hall',),
 ('Shirley Allen',),
 ('Cynthia Young',),
 ('Angela Hernandez',),
 ('Melissa King',),
 ('Brenda Wright',),
 ('Amy Lopez',),
 ('Anna Hill',),
 ('Rebecca Scott',),
 ('Virginia Green',),
 ('Kathleen Adams',),
 ('Pamela Baker',),
 ('Martha Gonzalez',),
 ('Debra Nelson',),
 ('Amanda Carter',),
 ('Stephanie Mitchell',),
 ('Carolyn Perez',),
 ('Christine Roberts',),
 ('Marie Turner',),
 ('Janet Phillips',),
 ('Catherine Campbell',),
 ('Fr

In [111]:
# grab the length of a string
get_info("""
    select first_name, last_name, length(first_name), length(last_name) from customer;
""")

[('Jared', 'Ely', 5, 3),
 ('Mary', 'Smith', 4, 5),
 ('Patricia', 'Johnson', 8, 7),
 ('Linda', 'Williams', 5, 8),
 ('Barbara', 'Jones', 7, 5),
 ('Elizabeth', 'Brown', 9, 5),
 ('Jennifer', 'Davis', 8, 5),
 ('Maria', 'Miller', 5, 6),
 ('Susan', 'Wilson', 5, 6),
 ('Margaret', 'Moore', 8, 5),
 ('Dorothy', 'Taylor', 7, 6),
 ('Lisa', 'Anderson', 4, 8),
 ('Nancy', 'Thomas', 5, 6),
 ('Karen', 'Jackson', 5, 7),
 ('Betty', 'White', 5, 5),
 ('Helen', 'Harris', 5, 6),
 ('Sandra', 'Martin', 6, 6),
 ('Donna', 'Thompson', 5, 8),
 ('Carol', 'Garcia', 5, 6),
 ('Ruth', 'Martinez', 4, 8),
 ('Sharon', 'Robinson', 6, 8),
 ('Michelle', 'Clark', 8, 5),
 ('Laura', 'Rodriguez', 5, 9),
 ('Sarah', 'Lewis', 5, 5),
 ('Kimberly', 'Lee', 8, 3),
 ('Deborah', 'Walker', 7, 6),
 ('Jessica', 'Hall', 7, 4),
 ('Shirley', 'Allen', 7, 5),
 ('Cynthia', 'Young', 7, 5),
 ('Angela', 'Hernandez', 6, 9),
 ('Melissa', 'King', 7, 4),
 ('Brenda', 'Wright', 6, 6),
 ('Amy', 'Lopez', 3, 5),
 ('Anna', 'Hill', 4, 4),
 ('Rebecca', 'Scott', 

In [113]:
# create an email for a customer
get_info("""
    select first_name, last_name, first_name || '.' || last_name || '@gmail.com' as email from customer limit 5;
""")

[('Jared', 'Ely', 'Jared.Ely@gmail.com'),
 ('Mary', 'Smith', 'Mary.Smith@gmail.com'),
 ('Patricia', 'Johnson', 'Patricia.Johnson@gmail.com'),
 ('Linda', 'Williams', 'Linda.Williams@gmail.com'),
 ('Barbara', 'Jones', 'Barbara.Jones@gmail.com')]

In [116]:
# custom email
get_info("""
select 
    first_name, 
    last_name, 
    lower(left(first_name, 1)) || '.' || 
    lower(left(last_name, 1)) || 
    '@gmail.com' as email 
    from customer limit 5;
""")

[('Jared', 'Ely', 'j.e@gmail.com'),
 ('Mary', 'Smith', 'm.s@gmail.com'),
 ('Patricia', 'Johnson', 'p.j@gmail.com'),
 ('Linda', 'Williams', 'l.w@gmail.com'),
 ('Barbara', 'Jones', 'b.j@gmail.com')]